In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from litQeval.eval_utils import *
import plotly.express as px
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [15]:
baseline = 'Crop Yield Prediction'
predicted = """
"compliant materials" OR (soft robotics) OR "soft actuators" OR "rehabilitation robotics" OR "pneumatic actuators" OR "robotic locomotion" OR "soft robotic systems" OR "soft robot control" OR "multi-material printing" OR "bio-inspired robotics" OR "continuum robots" OR "soft robot fabrication" OR "shape-morphing structures" OR "energy-efficient robotics" OR "soft robotic grippers" OR "soft exoskeletons" OR "soft robotic arms" OR "soft wearable robots" OR "autonomous soft robots" OR 
(("material properties" OR "adaptive control" OR "sensor integration" OR "flexible materials" OR "human-robot interaction" OR "deformation mechanics" OR "lightweight structures" OR "soft sensors" OR "artificial muscles" OR "robotic manipulation") AND (Robot OR Soft))
"""

data = get_data(baseline, predicted)
core_pubs = data["core_pubs"]
core_mean_embedding = data["core_mean_embedding"]
baseline_pubs = data["baseline_pubs"]
predicted_pubs = data["predicted_pubs"]
baseline_vs = data["baseline_vs"]
predicted_vs = data["predicted_vs"]
core_vs = data["core_vs"]
predicted_embeddings = np.array([embedding for embedding in predicted_vs.get(include=["embeddings"])["embeddings"]])
baseline_embeddings = np.array([embedding for embedding in baseline_vs.get(include=["embeddings"])["embeddings"]])
core_embeddings = np.squeeze([core_vs.get(i,include=["embeddings"])["embeddings"] for i in core_pubs])
core_mean_embedding.reshape(1, -1).shape, predicted_embeddings.shape, baseline_embeddings.shape


((1, 1536), (47993, 1536), (17573, 1536))

In [16]:
# predicted
cosine_sim = cosine_similarity(core_mean_embedding, predicted_embeddings).flatten()
predicted_pubs["similarity"] = cosine_sim

core_pubs_in_predicted = predicted_pubs[predicted_pubs["id"].isin(core_pubs)]
threshold = core_pubs_in_predicted["similarity"].min()
relevant_predicted_pubs = predicted_pubs[predicted_pubs["similarity"] >= threshold].copy()
relevant_predicted_pubs.shape[0] 

10161

In [17]:
# baseline
cosine_sim = cosine_similarity(core_mean_embedding, baseline_embeddings).flatten()
baseline_pubs["similarity"] = cosine_sim

core_pubs_in_baseline = baseline_pubs[baseline_pubs["id"].isin(core_pubs)]
threshold = core_pubs_in_baseline["similarity"].min()
relevent_baseline_pubs = baseline_pubs[baseline_pubs["similarity"] >= threshold].copy()
relevent_baseline_pubs.shape[0]

7946

### Cosine Similarity Measure

In [18]:
recall = evaluate_recall(core_pubs, baseline_pubs, predicted_pubs)
# semnatic precision: every element that is more similar than the least similar core publication is considered relevant
# relevant_predicted_pubs: publications that are more similar than the least similar core publication.
pred_precision = relevant_predicted_pubs.shape[0] / predicted_pubs.shape[0] # total number of found publications
baseline_precision = (relevent_baseline_pubs.shape[0] / baseline_pubs.shape[0]) if baseline_pubs.shape[0] > 0 else 0
pred_f2 = fscore(pred_precision, recall["predicted_recall"], 2)
baseline_f2 = fscore(baseline_precision, recall["baseline_recall"], 2)
df = pd.DataFrame({
    "Semantic Precision": [pred_precision, baseline_precision],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic F2": [pred_f2, baseline_f2]
}, index=["Predicted", "Baseline"])
df

,Semantic Precision,Recall,Semantic F2
Predicted,0.211718,0.722222,0.487248
Baseline,0.452171,0.555556,0.531262


### Minimum Volume Enclosing Ellipsoid MMVE

In [19]:
import numpy.linalg as la

def mvee(points, tol=0.0001):
    """
    Finds the ellipse equation in "center form"
    (x-c).T * A * (x-c) = 1
    """
    N, d = points.shape
    Q = np.column_stack((points, np.ones(N))).T
    err = tol+1.0
    u = np.ones(N)/N
    while err > tol:
        # assert u.sum() == 1 # invariant
        X = np.dot(np.dot(Q, np.diag(u)), Q.T)
        M = np.diag(np.dot(np.dot(Q.T, la.inv(X)), Q))
        jdx = np.argmax(M)
        step_size = (M[jdx]-d-1.0)/((d+1)*(M[jdx]-1.0))
        new_u = (1-step_size)*u
        new_u[jdx] += step_size
        err = la.norm(new_u-u)
        u = new_u
    c = np.dot(u, points)
    A = la.inv(np.dot(np.dot(points.T, np.diag(u)), points)
               - np.multiply.outer(c, c))/d
    return A, c

A, c = mvee(core_embeddings)

In [20]:
base_is_inside = is_inside_ellipse(A, c, baseline_embeddings)
predicted_is_inside = is_inside_ellipse(A, c, predicted_embeddings)

100%|██████████| 47993/47993 [01:20<00:00, 592.66it/s]


In [31]:
mvve_prec_baseline = base_is_inside.sum() / len(base_is_inside)
mvve_prec_predicted = predicted_is_inside.sum() / len(predicted_is_inside)

mvve_df = pd.DataFrame({
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])
mvve_df

,MVVE Precision,Recall,MVVE F2
Predicted,0.450545,0.722222,0.644496
Baseline,0.491948,0.555556,0.541551


In [38]:
results = pd.DataFrame({
    "Query": [predicted] + [baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic Precision": [pred_precision, baseline_precision],
    "Semantic F2": [pred_f2, baseline_f2],
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])

try:
    old_results = pd.read_excel("results.xlsx", index_col=0)
    results = pd.concat([old_results, results]).drop_duplicates(subset=["Query"]).round(3)
    results.to_excel("results.xlsx")
except FileNotFoundError:
    results.to_excel("results.xlsx")

display(results)

,Query,Recall,Semantic Precision,Semantic F2,MVVE Precision,MVVE F2
Predicted,"\n""spinal stenosis"" OR ""spinal surgery risks"" ...",0.723,0.316,0.575,0.438,0.640
Baseline,"""Cervical Myelopathy""",0.277,0.523,0.305,0.489,0.303
Predicted,"\n""geospatial data"" OR ""aerial photography"" OR...",0.120,0.036,0.082,0.518,0.142
Baseline,"""Drones in Agriculture""",0.000,0.000,0.000,0.592,0.000
Predicted,"\n""digital surgery"" OR ""surgical navigation"" O...",0.957,0.061,0.244,0.330,0.694
Baseline,Robotic Arthroplasty,0.957,0.595,0.853,0.401,0.749
Predicted,"\n""compliant materials"" OR (soft robotics) OR ...",0.722,0.212,0.487,0.451,0.644
Baseline,Soft Robotics,0.556,0.452,0.531,0.492,0.542
